In [1]:
from PIL import Image
import itertools
import numpy as np
import pandas as pd
import warnings
import nltk
import math
import time
import re
import os
import pickle
#Deep-Learning Library
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

Using TensorFlow backend.


In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = "/content/drive/My Drive/Major Project B.Tech Sem 8/Breast Cancer Dataset/"

In [0]:
train_data_dir=path+"Train"
test_data_dir=path+"Test"

#Training paths
data_cnn_VGG16_train=path+"VGG16/train/data_cnn_VGG16_train.npy"
data_filenames_VGG16_train=path+"VGG16/train/data_filenames_VGG16_train.npy"
data_cancerclass_VGG16_train=path+"VGG16/train/data_cancerclass_VGG16_train.npy"
data_cancertype_VGG16_train=path+"VGG16/train/data_cancertype_VGG16_train.npy"
data_mag_VGG16_train=path+"VGG16/train/data_mag_VGG16_train.npy"

#Training paths
data_cnn_VGG16_test=path+"VGG16/train/data_cnn_VGG16_test.npy"
data_filenames_VGG16_test=path+"VGG16/train/data_filenames_VGG16_test.npy"
data_cancerclass_VGG16_test=path+"VGG16/train/data_cancerclass_VGG16_test.npy"
data_cancertype_VGG16_test=path+"VGG16/train/data_cancertype_VGG16_test.npy"
data_mag_VGG16_test=path+"VGG16/train/data_mag_VGG16_test.npy"

In [29]:
start_time = time.clock()
# dimensions of our images.
img_width, img_height = 224, 224
top_model_weights_path = 'bottleneck_fc_model.h5'
epochs = 50
batch_size = 1
print("Started")

def save_bottlebeck_features(train_data=train_data_dir,nb_train_samples=5000,batch_size=1):
    
    #Function to compute VGG-16 CNN for image feature extraction.
        
    filenames = []
    cancerclass=[]
    cancertype=[]
    mag=[]
    datagen = ImageDataGenerator(rescale=1. / 255)
    print("Initialized Required data and datagen done")                                                
    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    print("Model settled")
    generator = datagen.flow_from_directory(
        train_data,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    print("Genrator work finish")
    start_time = time.clock()
    for i in generator.filenames:
        
        # Array for File NAmes
        filenames.append(i)
        
        # Array For Cancer Class
        X=re.search("_B_", i)
        if (X):
          cancer=1
          cancerclass.append(cancer)
        else:
          cancer=2
          cancerclass.append(cancer)
        
        # Array For Cancer Type
        pattern=re.compile('\w([AFTPDLMC]*)-\w')
        cancersubtypes=['_A-','_F-','_PT-','_TA-','_DC-','_LC-','_MC-','_PC-']
        codeforcancersubtypes=[11,12,13,14,21,22,23,24]
        
        for types in cancersubtypes:
          k=0
          obtainedpattern=pattern.match(i)
          print(type(i))
          
          #print(type(pattern.match(i)))
          #print(obtainedpattern)
          if(obtainedpattern):
            cancertype.append(codeforcancersubtypes[k])
            print(i,types)
          k=k+1
            
        """if(re.search("_A-", i)):
          cancerclass="11"
          cancertype.append(cancerclass)
        elif(re.search("_F-", i)):
          cancerclass="12"
          cancertype.append(cancerclass)
        elif(re.search("_PT-", i)):
          cancerclass="13"
          cancertype.append(cancerclass)
        elif(re.search("_TA-", i)):
          cancerclass="14"
          cancertype.append(cancerclass)        
        elif(re.search("_DC-", i)):
          cancerclass="21"
          cancertype.append(cancerclass)
        elif(re.search("_LC-", i)):
          cancerclass="22"
          cancertype.append(cancerclass)
        elif(re.search("_MC-", i)):
          cancerclass="23"
          cancertype.append(cancerclass)
        else:
          cancerclass="24"
          cancertype.append(cancerclass)
        
        # Array for Magnification
        if(re.search("-40-", i)):
          magni="40"
          mag.append(magni)
        elif(re.search("-100-", i)):
          magni="100"
          mag.append(magni)
        elif(re.search("-200-", i)):
          magni="200"
          mag.append(magni)
        elif(re.search("-400-", i)):
          magni="400"
          mag.append(magni)
        """
    print(cancerclass)    
    print(cancertype)
    """print("For loop ended")
    print(time.clock() - start_time, "seconds, time for loop")
    print(generator)
    print("training start")
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
    print("training finised")
    print("reshape the trained vector")
    bottleneck_features_train = bottleneck_features_train.reshape((5000,25088))
    print("Trainning done")
    np.save(open(data_cnn_VGG16_train, 'wb'), bottleneck_features_train)
    np.save(open(data_filenames_VGG16_train, 'wb'), np.array(filenames))
    np.save(open(data_cancerclass_VGG16_train, 'wb'), np.array(cancerclass))
    np.save(open(data_cancertype_VGG16_train, 'wb'), np.array(cancertype))
    np.save(open(data_mog_VGG16_train, 'wb'), np.array(mag))
    print("npy file saved")"""
    

save_bottlebeck_features()

print(time.clock() - start_time, "seconds")

Started
Initialized Required data and datagen done
Model settled
Found 5000 images belonging to 1 classes.
Genrator work finish
<class '_sre.SRE_Pattern'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class '_sre.SRE_Pattern'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class '_sre.SRE_Pattern'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class '_sre.SRE_Pattern'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class '_sre.SRE_Pattern'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class '_sre.SRE_Pattern'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class '_sre.SRE_Pattern'>
<class 'str